# **Homework 3 Solution**

In this homework, we'll implement an end to end semantic search engine

> It's possible that your answers won't match exactly. If it's the case, select the closest one.

## Q1. Prepare Documents

Import documents.json, read the file and prepare the dataset:

```bash
import json

with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

len(documents)
```

### Q1:


How many records we have in the pre-processed "documents"?
* 1000
* 1051
* 901
* 948

### A1:

In [1]:
import json

with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

len(documents)

948

Import sentence transformer library. Please review the Sentence Transformer pretrained documentation here: https://www.sbert.net/docs/sentence_transformer/pretrained_models.html#model-overview

```bash
# This is a new library compared to the previous modules. 
# Please perform "pip install sentence_transformers==2.7.0"

from sentence_transformers import SentenceTransformer

# if you get an error do the following:
# 1. Uninstall numpy 
# 2. Uninstall torch
# 3. pip install numpy==1.26.4
# 4. pip install torch
# run the above cell, it should work
model = SentenceTransformer("all-MiniLM-L12-v2")
```

### Q2:

What is the model size (in MB) and average performance?

* [420, 63.30]
* [120, 59.76]
* [290, 59.84]
* [420, 51.72]

### A2:

In [2]:

from sentence_transformers import SentenceTransformer

# if you get an error do the following:
# 1. Uninstall numpy 
# 2. Uninstall torch
# 3. pip install numpy==1.26.4
# 4. pip install torch
# run the above cell, it should work
model = SentenceTransformer("all-MiniLM-L12-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\Odiaka\miniconda3\envs\alexey\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Odiaka\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L12-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

120MB,59.76

```bash
len(model.encode("This is a simple sentence"))
```

### Q3:


What is the dimension of the model embedding?

* 768
* 265
* 384
* 1056

### A3:

In [3]:
len(model.encode("This is a simple sentence"))

384


384.

```bash
#created the dense vector using the pre-trained model
operations = []
for doc in documents:
    # Transforming the title into an embedding using the model
    doc["question_vector"] = model.encode(doc["question"]).tolist()
    operations.append(doc)
```


Establish connection to Elasticsearch 

```bash
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

es_client.info()
```

## Q4: Create Mappings and Index

In the mappings, change "section" to "keyword" type

```bash
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "keyword"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "question_vector":{"type":"dense_vector","dims": 384,"index":True,"similarity": "cosine"
        },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)
```

Add documents into index

```bash
for doc in operations:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)
```

In the es_client.indices.delete statement, what does "ignore_unavailable" mean?
* If the pre-trained model in unavailable, skip this statement
* If the index is unavailable, skip this statement
* If the pre-trained model in unavailable, don't skip this statement
* If the index is unavailable, don't skip this statement


In [4]:
#created the dense vector using the pre-trained model
operations = []
for doc in documents:
    # Transforming the title into an embedding using the model
    doc["question_vector"] = model.encode(doc["question"]).tolist()
    operations.append(doc)

In [5]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

es_client.info()

ObjectApiResponse({'name': 'a057d5e78121', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'K7EmkH9MRGW2ZIodXcVqHg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [6]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "keyword"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "question_vector":{"type":"dense_vector","dims": 384,"index":True,"similarity": "cosine"
        },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [7]:
for doc in operations:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

### A4:

If the index is unavailable, skip this statement

### Q5: 

Use the search term "how to enrol to course?" and perform semantic search

```bash
search_term = "how to enrol to course?"
vector_search_term = model.encode(search_term)

query = {
    "field" : "question_vector",
    "query_vector" :  vector_search_term,
    "k" : 5,
    "num_candidates" : 10000, 
}

res = es_client.search(index=index_name, knn=query,source=["text","section","question","course"])
res["hits"]["hits"]
```
What is the similarity score, section and course for the first result?

* [0.78, 'Module 1: Introduction', 'mlops-zoomcamp']
* [0.74, 'General course-related questions', 'data-engineering-zoomcamp']
* [0.72, 'Projects (Midterm and Capstone)', 'machine-learning-zoomcamp']
* [0.71, 'General course-related questions', 'machine-learning-zoomcamp']

### A5:

In [8]:
search_term = "how to enrol to course?"
vector_search_term = model.encode(search_term)

query = {
    "field" : "question_vector",
    "query_vector" :  vector_search_term,
    "k" : 5,
    "num_candidates" : 10000, 
}

res = es_client.search(index=index_name, knn=query,source=["text","section","question","course"])
res["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'Nk86hZABvDJOFodXi85b',
  '_score': 0.8520479,
  '_source': {'question': 'Course - What are the prerequisites for this course?',
   'course': 'data-engineering-zoomcamp',
   'section': 'General course-related questions',
   'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites'}},
 {'_index': 'course-questions',
  '_id': 'V086hZABvDJOFodXjM6-',
  '_score': 0.8381673,
  '_source': {'question': 'How can we contribute to the course?',
   'course': 'data-engineering-zoomcamp',
   'section': 'General course-related questions',
   'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.'}},
 {'_index': 'course-questions',
  '_id': '-U86hZABvDJOFodXnM-3',
  '_score': 0.8284229,
  '_source': {'question': 'I just joined. What should I do next? How can I access course materials?',
   'course': 'machine-learning-zoomcamp',
   'section': 'Gen

* 'similarity_score': 0.8520479,
* 'course': 'data-engineering-zoomcamp',
* 'section': 'General course-related questions'

[0.74, 'General course-related questions', 'data-engineering-zoomcamp']

### Q6: Perform Semantic Search & Filtering

Filter the results to "General course-related questions" section only

```bash
knn_query = {
    "field" : "text_vector",
    "query_vector" :  vector_search_term,
    "k" : 5,
    "num_candidates" : 10000
}

response = es_client.search(
    index=index_name,
    query={
        "match": {
                "section": "General course-related questions"
            },
        },
        
    knn=knn_query,
    size=5
)

response["hits"]["hits"]
```
Do you see the results filtered only to "General course-related questions" section?

* Yes
* No

### A6:

In [10]:
knn_query = {
    "field" : "question_vector",
    "query_vector" :  vector_search_term,
    "k" : 5,
    "num_candidates" : 10000
}

response = es_client.search(
    index=index_name,
    query={
        "match": {
                "section": "General course-related questions"
            },
        },
        
    knn=knn_query,
    size=5
)

response["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'Nk86hZABvDJOFodXi85b',
  '_score': 3.4953291,
  '_source': {'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
   'section': 'General course-related questions',
   'question': 'Course - What are the prerequisites for this course?',
   'course': 'data-engineering-zoomcamp',
   'question_vector': [0.04312111809849739,
    0.05837031453847885,
    0.005022975616157055,
    0.028570590540766716,
    -0.022065501660108566,
    0.004731936380267143,
    -0.025459880009293556,
    -0.0008887068834155798,
    -0.16603535413742065,
    -0.004694763105362654,
    -0.05639379471540451,
    -0.10678312927484512,
    -0.06021314114332199,
    0.03971245139837265,
    -0.06361556798219681,
    0.03923669457435608,
    -0.004405470564961433,
    -0.008392641320824623,
    0.14483699202537537,
    -0.07729385048151016,
    0.015076633542776108,
    0.020861979573965073,
    0.019407764077186584,
    -0.031069425866007805,
    0.007932012

Yes, I do.